<a href="https://colab.research.google.com/github/echu06/SQL_practice/blob/main/SQLday_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Queries

SQL stands for "structured query language". It is a standardized language going back to 1982. For the purposes of data retrieval, the main command we will use is "SELECT":

![](https://www.w3resource.com/w3r_images/select-syntax.gif)

## Plan for the workshop

* Day 1 - row filtering, group by
* Day 2 - data joins, arrays and UNNEST
* Day 3 - window functions
* Day 4 - dashboards (Looker, Dash)
* Day 5 - Querying Wikidata using SPARQL (https://www.wikidata.org/wiki/Wikidata:SPARQL_tutorial)

## SQL inside Google Sheets

As a warm up, we will start with doing simple queries inside Google Sheets.

We will use this sheet: https://docs.google.com/spreadsheets/d/19ssb71XhlcBrWnoN6a-9702QnSmZzRpt-hnps-yRNwk/edit?usp=sharing (and later another one, at this [link](https://docs.google.com/spreadsheets/d/1FRgX6DR9SRMg2eD_M1Xbl6MFGgaWZfe8XSh5PYDPjJ0/edit?usp=sharing))

Sheets have a function `QUERY` that will execute a query in simplified SQL:https://support.google.com/docs/answer/3093343?hl=en

Specifics of this version of SQL are described here: https://developers.google.com/chart/interactive/docs/querylanguage

Notice that the IMDB tables do not have any information whether the item is a Netflix show or not. We are searching only using the title (column `primaryTitle`)


The important point is that you can only use the "native" column names, like "A", "B", etc. - not the names that are provided in the header row.

The "from xxx" part is eliminated, because the function takes a cell range as an input.

The query can work only with one table/range - there are no inner joins.


## Example 1 - Netflix shows of 2024

This sheet: [link](https://docs.google.com/spreadsheets/d/19ssb71XhlcBrWnoN6a-9702QnSmZzRpt-hnps-yRNwk/edit?usp=sharing) contains data from IMDB and TMDB. The main focus is the statistics on the ratings of episodes of shows. The table was updated daily, and we have the date as `import_date`.

---

Which shows contain the word 'Last' in their name?

```
=UNIQUE(QUERY(Sheet1!A:L, "select B where B contains 'Last'"))
```

The `contains` operator is case sensitive. We can use a scalar function `upper` to convert the show name to uppercase, and then compare against the 'LAST' string.

```
=UNIQUE(QUERY(Sheet1!A:L, "select B where upper(B) contains 'LAST'"))
```


---

Get the IMDB ratings for the first episode of "Miss Night and Day":

```
=QUERY(Sheet1!A:L, "select * where upper(B) = 'MISS NIGHT AND DAY' and I = 1 and J = 1")
```

Now, let's also order them by `import_date`:

```
=QUERY(Sheet1!A:L, "select * where upper(B) = 'MISS NIGHT AND DAY' and I = 1 and J = 1 order by L")
```

---

### Aggregation

The `where` clause operates on the rows of a table. It selects the rows that meet a condition. This is useful when we use a database and use SQL to retrieve the rows that we need at the time.

We can also compute some aggregate statistics. The simplified SQL in Sheets supports `min`, `max`, `avg`, `sum`, and `count`. ([link](https://developers.google.com/chart/interactive/docs/querylanguage#aggregation-functions))

---

Which show had the most episodes?

```
=QUERY(Sheet1!A:L, "select B, max(J) group by B, I order by max(J) desc")
```

---

Which show had the most votes for an episode?

```
=QUERY(Sheet1!A:L, "select B, max(H) group by B order by max(H) desc")
```

Which show had the largest ratio of IMDB votes to TMDB votes? Let's select only those episodes that had more than 1,000 votes on IMDB.

```
=QUERY(Sheet1!A:L, "select B, max(H)/max(D) where H > 1000 group by B order by max(H)/max(D)")
```

The query above illustrates the point that you have seen in the image above, but did not have a chance to apprecite: the order of statements in a query. Specifically, the `where` clause goes before the `group by` clause.


---

### Pivot

Pivoting a table involves converting it into the "wide" format - contents of one column make up several columns after the transformation.

For instance, I want to compute the ratio of the IMDB ratings of season 1 episode 1 on '2024-05-01' for two shows: `3 Body Problem` and `A Killer Paradox`

Let's start by learning to filter the necessary rows:

```
=QUERY(Sheet1!A:L, "select B, H, L where I = 1 and J = 1 and (B = '3 Body Problem' or B = 'A Killer Paradox')")
```

Now, let's add the filter by `import_date`:

```
=QUERY(Sheet1!A:L, "select B, H, L where I = 1 and J = 1 and (B = '3 Body Problem' or B = 'A Killer Paradox') and L = date '2024-05-01'")
```

Our version of SQL does not have the lag function that would allow us to reference different rows, but it does allow us do operations row-wise - when the values are in the same row.

Thus, the way forward is to pivot the table, so that we have the number of votes as separate columns.

```
=QUERY(Sheet1!A:L,
"select max(H)
  where I = 1 and J = 1
  and (B = '3 Body Problem' or B = 'A Killer Paradox')
  and L = date '2024-05-01'
  pivot B")
```

One interesting twist is that pivoting requires using aggregation function. I had to provide `max(H)`. Because I had only one row for each show, `max(H)` was the same as just the value of `H` - the `imdb_num_of_votes`

### Other examples

These examples are based on the data in this [sheet](https://docs.google.com/spreadsheets/d/1FRgX6DR9SRMg2eD_M1Xbl6MFGgaWZfe8XSh5PYDPjJ0/edit?usp=sharing)


**List of towns that have `Big Y` stores:**

`=QUERY(Sheet2!A:V, "select E, O where E like 'BIG Y' order by O", 1)`

**List of shows/films that were created by HBO and then distributed by other networks:**

`=UNIQUE(QUERY(Sheet3!A:F, "select A, B, D, E, F where E = 'HBO' and E <> F and F <> '' order by E, B", 1))`

The query language in Google sheets does not have a "distinct" clause. To handle duplicate records, the result is enclosed into the sheet function `UNIQUE()`.

**Top 25 Enlish language films based on the number of weeks they stayed in Top 10 list**:

`=QUERY(Sheet4!A:E, "select D, count(D) where B = 'Films (English)' group by D order by count(D) desc limit 25", 1) `


---
---